In [18]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

$\large{\text{Foundations of Kinematics and Algorithms in Robotics}} \\ \large{\text{Module 2: Deep Dive into Orientation}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

### Contents

[1.0 Skew Symmetric Matrices](#1)  
[2.0 Lie Groups](#2)  
[3.0 Angle Axis Representation](#3)  
[4.0 The Special Orthogonal Group in Three Dimensions](#4)  
[5.0 Three Angle Representations](#5)  
[6.0 Quaternions](#6)

In [19]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

import spatialgeometry as sg

# The matrix exponential method
from scipy.linalg import expm, logm

# sympy provides symbolic mathematic operations 
import sympy as sym

# setting up sympy to print nicely
from IPython import display as ds
from sympy.physics.mechanics import init_vprinting
from sympy.physics.vector import vlatex
init_vprinting(use_latex='mathjax')

<br>

<a id='1'></a>
# 1.0 Skew Symmetric Matrices
---


We will start this Module with some required knowledge on skew-symmetric matrices.

## Skew Symmetric Matrices

In robotics, we often use skew symmetric matrices. These are a special type of matrix that have the following property

$$
    -\mathbf{A} = \mathbf{A}^\top.
$$

the negative of the matrix is equal to its transpose. This property means that the diagonal elements of the matrix are all zero due to the symmetry constraint, a skew-symmetric matrix $\in \mathbb{R}^{3 \times 3}$ can only contain three unique values.

If we have a vector $\mathbf{a} \in \mathbb{R}^3$

$$
    \mathbf{a} = (a_1,\ a_2,\ a_3)^\top.
$$

then the skew-symmetric matrix of $\mathbf{a}$ is

$$
    [\mathbf{a}]_\times =
    \begin{pmatrix}
        0 & -a_3 & a_2 \\
        a_3 & 0 & -a_1 \\
        -a_2 & a_1 & 0
    \end{pmatrix}.
$$

The skew symmetric matrix is a useful tool for representing the cross product of two vectors. The cross product of two vectors $\mathbf{a}$ and $\mathbf{b}$ is defined as

$$
    \mathbf{a} \times \mathbf{b} = [\mathbf{a}]_\times \mathbf{b}
$$

where $\mathbf{b} = (b_1,\ b_2,\ b_3)^\top$.



The vex operator $\mathsf{V}_\times(\cdot)$ is the inverse of the skew operator $[\cdot]_\times$. It takes a skew-symmetric matrix and returns the corresponding vector

$$
    \mathsf{V}_\times([\mathbf{a}]_\times) = \mathbf{a}.
$$

## Augmented Skew Symmetric Matrices

In robotics, we also use augmented skew-symmetric matrices, which are matrices that contain both a skew-symmetric matrix and a vector -- somewhat similar to a homogeneous transformation matrix containing a rotation matrix and a translation vector. The diagonal of the matrix are all zeroes.

An augmented skew-symmetric matrix $\in \mathbb{R}^{4 \times 4}$ can contain six unique values.

If we have a vector $\mathbf{a} \in \mathbb{R}^6$

$$
    \mathbf{a} = (a_1,\ a_2,\ a_3,\ a_4,\ a_5,\ a_6)^\top.
$$

then the augmented skew-symmetric matrix of $\mathbf{a}$ is

$$
    [\mathbf{a}] =
    \begin{pmatrix}
        0 & -a_3 & a_2 & a_4 \\
        a_3 & 0 & -a_1 & a_5 \\
        -a_2 & a_1 & 0 & a_6 \\
        0 & 0 & 0 & 0
    \end{pmatrix}.
$$

The augmented vex operator $\mathsf{V}(\cdot)$ is the inverse of the augmented skew operator $[\cdot]$. It takes an augmented skew-symmetric matrix and returns the corresponding vector

$$
    \mathsf{V}([\mathbf{a}]) = \mathbf{a}.
$$

## Conversions

<img src="Figures/2/skew_dark.png" alt="drawing" width="1000"/>

Shown on the left is a vector $\bf{s}\in \mathbb{R}^3$ along with its corresponding skew symmetric matrix $\bf{S} \in \bf{so}(3) \subset \mathbb{R}^{3 \times 3}$. Shown on the right is a vector $\bf{\hat{s}}\in \mathbb{R}^6$ along with its corresponding augmented skew symmetric matrix $\bf{\hat{S}} \in \bf{se}(3) \subset \mathbb{R}^{4 \times 4}$. The skew functions $[\cdot]_\times : \mathbb{R}^3 \mapsto \bf{so}(3)$ maps a vector to a skew symmetric matrix, and $[\cdot] : \mathbb{R}^6 \mapsto \bf{se}(3)$ maps a vector to an augmented skew symmetric matrix. The inverse skew functions $\mathsf{V}_\times(\cdot) : \bf{so}(3) \mapsto \mathbb{R}^3$ maps a skew symmetric matrix to a vector and $\mathsf{V}(\cdot) : \bf{se}(3) \mapsto \mathbb{R}^6$ maps an augmented skew symmetric matrix to a vector.

## Test your Understanding

#### Question 1

Write a Python method that takes a vector $\mathbf{a} \in \mathbb{R}^3$ and returns the corresponding skew-symmetric matrix.

#### Question 2

Write a Python method that takes a skew symmetric matrix $\mathbf{A} \in \mathbb{R}^{3 \times 3}$ and returns the corresponding vector.

#### Question 3

Is the following a valid skew symmetric matrix?

$$
    \mathbf{A} =
    \begin{pmatrix}
        0 & 1 & 0 \\
        1 & 0 & 0 \\
        0 & 0 & 0
    \end{pmatrix}.
$$

#### Question 4

Is the following a valid skew symmetric matrix?

$$
    \mathbf{A} =
    \begin{pmatrix}
        0 & 1 & 0 \\
        -1 & 0 & 0 \\
        0 & 0 & 0
    \end{pmatrix}.
$$

#### Question 5

Is the following a valid skew symmetric matrix?

$$
    \mathbf{A} =
    \begin{pmatrix}
        0 & 1 & 2 \\
        -1 & 0 & 0 \\
        0 & 0 & 0
    \end{pmatrix}.
$$

#### Question 6

Write a Python method that can test if a matrix is a valid skew symmetric matrix.

#### Question 7

Write a Python method that can test if a matrix is a valid augmented skew symmetric matrix.


<br>

<a id='2'></a>
# 2.0 Lie Groups
---


As we learned in Module 1, we can represent
a 2D rotation using the Special Orthogonal Group in two dimensions $\mathbf{R} \in \mathbf{SO}(2) \subset \mathbb{R}^{2 \times 2}$
and
a 3D rotation using the Special Orthogonal Group in three dimensions $\mathbf{R} \in \mathbf{SO}(3) \subset \mathbb{R}^{3 \times 3}$.

These groups form __manifolds__. Think of a manifold as a surface that exists in $n$-dimensional space. For example, the points on a surface of a sphere form a manifold in 3D space. In the case of $\mathbf{SO}(2)$ matrices, they form a  manifold in the 4-dimensional space of all $2 \ \times \ 2$ matrices. An $\mathbf{SO}(3)$ is a manifold that inhabits 9-dimensional space since the matrix has 9 elements. A __Lie Group__ is a group that is also a differentiable manifold -- this means that the surface is _smooth_.



## Visualise a Manifold with a Sphere

Since 9-dimensional space is quite hard to visualise, let's look at a 2-dimensional smooth surface existing in 3-dimensional space. The surface of the sphere is the set of all points such

\begin{align*}
    x^2 + y^2 + z^2 &= 1
\end{align*}

where the radius of the sphere is 1.

<img src="Figures/2/fig1-1.png" alt="drawing" max-width="700"/>

This Figure displays the unit sphere. The surface of the sphere is a smooth 2-dimensional differentiable surface. This illustrates the concept of constraining a higher dimensional space to produce a lower dimensional _surface_ through constraints or a system of equations. In this case, the only points in 3-dimensional space that exist in the manifold are those that satisfy the equation $x^2 + y^2 + z^2 = 1$. The same principle applies to the $\mathbf{SO}(2)$ and $\mathbf{SO}(3)$ groups. It is in essence a subset of all possible points.

## Lie Algebra and the Tangent Space

At any point on the manifold we can construct tangent vectors. The set of all tangent vectors at that point form a vector space -- the tangent space. This is the multidimensional equivalent to a tangent line on a curve, or a tangent plane on a sphere. We can think of this as the set of all possible derivatives of the manifold at that point.

The tangent space of $\bf{SO}(3)$ lies in $\bf{so}(3)$ which are the Lie algebras.

The tangent space at the identity is described by the Lie algebra of the group, and the basis directions of the tangent space are called the generators of the group. There are three generators of $\bf{SO}(3)$ each belonging to $\bf{so}(3)$.

The three generators each provide a basis – which can be thought of as the $x$, $y$, and $z$ axes – for the group $\bf{SO}(3)$.

The generator for the x-axis is
\begin{equation*}
    g_x =
    \begin{pmatrix}
        0 & 0 & 0 \\
        0 & 0 & -1 \\
        0 & 1 & 0
    \end{pmatrix}.
\end{equation*}

The generator for the y-axis is
\begin{equation*}
    g_y =
    \begin{pmatrix}
        0 & 0 & 1 \\
        0 & 0 & 0 \\
        -1 & 0 & 0
    \end{pmatrix}.
\end{equation*}

The generator for the z-axis is
\begin{equation*}
    g_z =
    \begin{pmatrix}
        0 & -1 & 0 \\
        1 & 0 & 0 \\
        0 & 0 & 0
    \end{pmatrix}.
\end{equation*}

Every point in the tangent space $\bf{so}(3)$ can be expressed as a linear combination of basis matrices

\begin{equation*}
    \Omega(\eta_1, \ \eta_2, \ \eta_3) =
    \eta_1 \ g_x +
    \eta_2 \ g_y +
    \eta_3 \ g_z
\end{equation*}
where $\bf{\eta} = (\eta_1, \ \eta_2, \ \eta_3)$ are the exponential coordinates (also known as the Euler vector).

These generator matrices are each skew symmetric matrices. Lets make each of these in Python using the `skew` method provided by `spatialmath`

In [20]:
# Lets make the basis matrices in Python
gx = smb.skew([1, 0, 0])
gy = smb.skew([0, 1, 0])
gz = smb.skew([0, 0, 1])

# Visualise the results
ds.display(ds.Math(f"g_x = {sym.latex(sym.Matrix(gx))}"))
ds.display(ds.Math(f"g_y = {sym.latex(sym.Matrix(gy))}"))
ds.display(ds.Math(f"g_z = {sym.latex(sym.Matrix(gz))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Test your Understanding

#### Question 1

How many numbers are in the Euler vector?

#### Question 2

What does the Euler vector represent?

#### Question 3

What does the tangent space represent?



<br>

<a id='3'></a>
# 3.0 Angle Axis Representation
---


Together, the exponential coordinates define an axis and an angle

\begin{equation*}
    \bf{\eta} =
    \hat{\eta} \ \theta
\end{equation*}

where $\hat{\eta} \in \mathbb{R}^3$ is a unit vector which defines the axis and $\theta$ defines the angle or rotation around the axis.

Therefore $\eta$ describes four numbers with
\begin{equation*}
    \theta = \lvert\lvert \eta \rvert\rvert
\end{equation*}
and 
\begin{equation*}
    \hat{\eta} = \dfrac{\eta}{\lvert\lvert \eta \rvert\rvert}
\end{equation*}



The Figure below shows the axis and angle representation of a rotation

<img src="Figures/2/fig1-2.png" alt="drawing" width="400"/>

We have the following relationship between the Lie algebra $\phi$ and Lie group $\Phi$

\begin{equation*}
    \forall \bf{X} \in \bf{\phi} \Rightarrow e^{\bf{X}} \in \bf{\Phi}
\end{equation*}

which says that the exponential of any member of $\bf{so}(3)$ is a valid member of $\bf{SO}(3)$

Lets put this to the test in Python.

We will use the Spatialmath Package to create an $\bf{SO}(3)$ matrix that represents a rotation around x-axis by an amount $\pi$

In [38]:
rot_x = smb.rotx(np.pi)

# View the result
ds.display(ds.Math(f"R_x(\pi) = {sym.latex(sym.Matrix(np.round(rot_x, 2)))}"))


<IPython.core.display.Math object>

Now we will get the same rotation using the theory detailed out above

We will perform the equation
\begin{align*}
    R_x(\theta) &= e^{\theta \ g_x} \\
    R_x(\pi) &= e^{\pi \ g_x}
\end{align*}

In [22]:
rot_x = expm(np.pi * gx)

# View the result
ds.display(ds.Math(f"R_x(\pi) = {sym.latex(sym.Matrix(np.round(rot_x, 2)))}"))

<IPython.core.display.Math object>

which shows that we get the same result.

## Test your Understanding

#### Question 1

Make a Python method that creates an $\mathbf{SO}(3)$ matrix around the y-axis by an angle $\theta$ using the `expm` method.

#### Question 2

What is the difference between the Euler vector and the Angle Axis representation?

#### Question 3

Can we make any rotation using the Angle Axis representation?



<br>

<a id='4'></a>
# 4.0 The Special Orthogonal Group in Three Dimensions
---

We can create every rotation in $\bf{SO}(3)$ with the generators

\begin{align*}
    \Omega(\bf\eta)
    &= 
    \begin{pmatrix}
        0 & -\eta_3 & \eta_2 \\
        \eta_3 & 0 & -\eta_1 \\
        -\eta_2 & \eta_1 & 0
    \end{pmatrix} \\
    &= 
    [\eta]_\times
\end{align*}
where $[\cdot]_\times : \mathbb{R}^3 \mapsto \bf{so}(3)$ maps a vector to a skew symmetric matrix.

We then create a rotation with

\begin{align*}
    \bf{R}(\bf\eta)
    &= e^{[\eta]_\times} \\
    &= e^{[\hat{\eta}]_\times \ \theta} \\
\end{align*}


Coming up with the axis and angle to define $\eta$ can be tricky in practice. Another strategy can be to construct rotations using a combination of the generators.

For example, we can construct a rotation made rotations about the $x$-, $y$-, and $z$-axis

\begin{align*}
    \bf{R}(\alpha, \ \beta, \ \gamma)
    &= 
    e^{\gamma \ g_x} \ e^{\beta \ g_y} \ e^{\alpha \ g_y}
\end{align*}

In [23]:
rot_rpy = sm.SO3.Rx(np.pi/2).A @ sm.SO3.Ry(np.pi/4).A @ sm.SO3.Rz(np.pi/2).A

# View the result
ds.display(ds.Math(f"R = {sym.latex(sym.Matrix(np.round(rot_rpy, 2)))}"))

<IPython.core.display.Math object>

Now lets try using the generators

In [24]:
rot_rpy = expm(np.pi/2 * gx) @ expm(np.pi/4 * gy) @ expm(np.pi/2 * gz)

# View the result
ds.display(ds.Math(f"R = {sym.latex(sym.Matrix(np.round(rot_rpy, 2)))}"))

<IPython.core.display.Math object>

We can also go in the reverse direction and decompose a rotation into a combination of the generators using the matrix logarithm

In [25]:
sk_eta = logm(rot_rpy)

# View the result
ds.display(ds.Math(f"[\eta]_\\times = {sym.latex(sym.Matrix(np.round(sk_eta, 2)))}"))

<IPython.core.display.Math object>

An the the vex method we can get the Euler vector

In [26]:
eta = smb.vex(sk_eta)

# View the result
ds.display(ds.Math(f"\eta = {sym.latex(sym.Matrix(np.round(eta, 2)))}"))

<IPython.core.display.Math object>

and finally, we can decompose the Euler vector into the angle-axis representation

In [27]:
theta = np.linalg.norm(eta)
eta_hat = eta / np.linalg.norm(eta)

# View the result
ds.display(ds.Math(f"\hat{{\eta}} = {sym.latex(sym.Matrix(np.round(eta_hat, 2)))}"))
ds.display(ds.Math(f"\\theta = {sym.latex((np.round(theta, 2)))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## The $\mathbf{SO}(3)$ Elementary Rotations

In Module 1, we introduced the following elementary rotations for $\mathbf{SO}(3)$

$$
    \bf{R}_x(\theta) = 
    \left[\begin{matrix}1 & 0 & 0\\0 & \cos{\left(\theta \right)} & - \sin{\left(\theta \right)}\\0 & \sin{\left(\theta \right)} & \cos{\left(\theta \right)}\end{matrix}\right]
$$

$$
    \bf{R}_y(\theta) = 
    \left[\begin{matrix}\cos{\left(\theta \right)} & 0 & \sin{\left(\theta \right)}\\0 & 1 & 0\\- \sin{\left(\theta \right)} & 0 & \cos{\left(\theta \right)}\end{matrix}\right]
$$

$$    
    \bf{R}_z(\theta) = 
    \left[\begin{matrix}\cos{\left(\theta \right)} & - \sin{\left(\theta \right)} & 0\\\sin{\left(\theta \right)} & \cos{\left(\theta \right)} & 0\\0 & 0 & 1\end{matrix}\right]
$$

However, we did not show how these were derived. Using `sympy`, we will create these elementary rotations using symbolic expressions.



In [28]:
# Make symbolic variables for the rotation angle
θ = sym.symbols('θ')

# Perform the symbolic matrix exponential for θ * gx_x
# This will create the R_x(θ) matrix
Rx = sym.Matrix(θ * gx).exp()

# Perform the symbolic matrix exponential for θ * gx_y
# This will create the R_y(θ) matrix
Ry = sym.Matrix(θ * gy).exp()

# Perform the symbolic matrix exponential for θ * gx_z
# This will create the R_z(θ) matrix
Rz = sym.Matrix(θ * gz).exp()

ds.display(ds.Math(f"R_x(θ) = {sym.latex(sym.simplify(Rx))}"))
ds.display(ds.Math(f"R_y(θ) = {sym.latex(sym.simplify(Ry))}"))
ds.display(ds.Math(f"R_z(θ) = {sym.latex(sym.simplify(Rz))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## What does a Rotation Matrix Represent?

If we can create a rotation matrix $\bf{R}$ using the matrix exponential of the angle-axis representation, then what does the resulting matrix represent?

The columns of $\bf{R}$ are the new basis vectors of the rotated frame. The rows of $\bf{R}$ are the basis vectors of the original frame relative to the new frame.

This means that the first column of $\bf{R}$ is the new $x$-axis in the rotated frame, the second column is the new $y$-axis, and the third column is the new $z$-axis.

Let's consider a two dimensional rotation as it's much easier to visualise.

We have a rotation matrix

$$
    \bf{R} = 
    \begin{pmatrix}
        0.866 & -0.5 \\
        0.5 & 0.866
    \end{pmatrix}
$$

that represents a rotation of $30^\circ$.

Looking at the first column of $\bf{R}$, we can define the first basis vector as

$$
    \hat{\mathbf{x}} = 
    \begin{pmatrix}
        0.866 \\
        0.5
    \end{pmatrix}
$$

and we can define the second basis vector as

$$
    \hat{\mathbf{y}} = 
    \begin{pmatrix}
        -0.5 \\
        0.866
    \end{pmatrix}
$$

We can then plot these basis vectors to see what the rotated coordinate frame looks like.


<img src="Figures/2/fig1-3.png" alt="drawing"/>

## Test your Understanding

#### Question 1

What is the most minimal representation of a rotation matrix?

#### Question 2

What is the advantage of a rotation matrix over an Euler vector or the angle-axis representation?

#### Question 3

What is the relationship between the columns of a rotation matrix and the rows of the inverse of the rotation matrix?

#### Question 4

What is the relationship between the columns of a rotation matrix and the rows of the transpose of the rotation matrix?

#### Question 5

Use `sympy` to generate the symbolic expression for the following rotation matrix (use the Python code above as a guide):

\begin{align*}
    \bf{R}(\alpha, \ \beta, \ \gamma)
    &= 
    e^{\gamma \ g_x} \ e^{\beta \ g_y} \ e^{\alpha \ g_y}
\end{align*}

#### Question 6

Why is it important for the columns of a rotation to be unit length and orthogonal?

#### Question 7

Why is it important for a rotation matrix to have a determinant of $+1$?


<br>

<a id='5'></a>
# 5.0 Three Angle Representations
---

Euler’s rotation theorem:


_Any two independent orthonormal coordinate frames can be related by a sequence of rotations (not more than three) about coordinate axes, where no two successive rotations may be about the same axis._

Euler’s rotation theorem means that a rotation between any two coordinate frames in 3D can be represented by a sequence of three rotation angles each associated with a particular coordinate frame axis. The rotations are applied consecutively: the first rotates the world frame {0} around some axis to create a new coordinate frame {1}; then a rotation about some axis of {1} results in frame {2}; and finally a rotation about some axis of {2} results in frame {3}.

There are a total of twelve unique rotation sequences. Six involve repetition, but not successive, of rotations about one particular axis:
* XYX
* XZX
* YXY
* YZY
* ZXZ
* ZYZ

Another six are characterized by rotations about all three axes:
* XYZ
* XZY
* YZX
* YXZ
* ZXY
* ZYX

## Euler Angles

If there are twelve possible sequences of three angle representations, then what are Euler angles?

Unfortunately, Euler angles are not a unique representation of a rotation and any of the twelve sequences may be referred to as Euler angles. Therefore, when specifying that Euler angles are used, it is important to also specify the sequence of rotations. Euler angles as a whole are not very useful to us, but two of the sequences are very useful.

## Roll-Pitch-Yaw Angles

You may be thinking that roll-pitch-yaw angles help us to specify the specific sequence of three-angle representation that are being used. However, this is not the case. Roll-pitch-yaw (RPY) angles may refer to either the XYZ or ZYX sequence depending on the field and application.

For ZYX RPY angles, we have

$$
    \mathbf{\Gamma} =
    \begin{pmatrix}
        \alpha \\
        \beta \\
        \gamma
    \end{pmatrix}
$$

and the corresponding rotation matrix is

$$
    \bf{R}(\mathbf{\Gamma})
    = 
    \bf{R}_z(\gamma) \ \bf{R}_y(\beta) \ \bf{R}_x(\alpha)
$$

and note that the order we apply the angles is the reverse order as shown in the angle vector $\mathbf{\Gamma}$.

For XYZ RPY angles, we have the same angle vector

$$
    \mathbf{\Gamma} =
    \begin{pmatrix}
        \alpha \\
        \beta \\
        \gamma
    \end{pmatrix}
$$

but a different equation to get the rotation matrix

$$
    \bf{R}(\mathbf{\Gamma})
    = 
    \bf{R}_x(\gamma) \ \bf{R}_y(\beta) \ \bf{R}_z(\alpha)
$$

## Why have so many Angle Representations?

Let's first consider a real-world application for RPY angles.

When describing the attitude of vehicles such as ships, aircraft and cars, the convention is that the x-axis of the body frame points in the forward direction and its z-axis points either up or down. We start with the world reference frame and in order:

* rotate about the world z-axis by the yaw angle, $\gamma$, so that the x-axis points in the direction of travel, then
* rotate about the y-axis of the frame above by the pitch angle, $\beta$, which sets the angle of the vehicle’s longitudinal axis relative to the horizontal plane (pitching the nose up or nose down), and then finally
* rotate about the x-axis of the frame above by the roll angle, $\alpha$, so that the vehicle’s body rolls about its longitudinal axis.

This is a very practical and achievable way of describing the attitude of a vehicle. A pilot can read the orientation of the aircraft through an instrument or report the orientation through comprehensible three numbers. This is much more convenient than reporting the nine numbers of a rotation matrix. Additionally, we can grasp the meaning of RPY angles immediately, whereas it is not so easy to understand the meaning of a rotation matrix, angle axis, or Euler vector without a visualisation tool or a conversion to Euler angles.

So you may wonder, why don't we just use RPY angles for everything?

Firstly, there are inherent flaws to the RPY angles, and Euler angles as a whole. They suffer from singularities and gimbal lock which will we discuss next.

Secondly, a roll-pitch-yaw angle is just that, a vector of angles. These angles do not form a group and therefore do not have a group operation. This means that we cannot add, multiple, divide or use some other operation to any two RPY angles to get a new RPY angle. When we want to compose two sets of RPY angles, we will typically first convert them to rotation matrices and then multiply them together before converting back to RPY angles.

## Singularities and Gimbal Lock

A fundamental problem with all the three-angle representations just described is singularity. This is also known as gimbal lock, a geeky term made famous in the movie Apollo 13. The term is related to mechanical gimbal systems.

Singularities occur with Euler angles when axes or rotation are aligned in a certain manner.

### What is a Singularity?

In mathematics, a singularity is a point in a function where the function is not defined or the mathematical object ceases to be well defined.

With a matrix, a singularity is a point where the matrix is not invertible, its determinant is zero, and its rank is less than the number of rows or columns.

In [29]:
# Make a non-singular matrix
A = np.eye(4)

# Make a singular matrix
B = np.array([
    [1, 2, 3, 4],
    [1, 4, 3, 8],
    [1, 6, 3, 12],
    [1, 8, 3, 16]
])

# Check if A is singular
print(f"Rank of A: {np.linalg.matrix_rank(A)}")
print(f"Determinant of A: {np.linalg.det(A)}")

# Check if B is singular
print(f"Rank of B: {np.linalg.matrix_rank(B)}")
print(f"Determinant of B: {np.linalg.det(B)}")

Rank of A: 4
Determinant of A: 1.0
Rank of B: 2
Determinant of B: 0.0


If you recall the intuitive meaning of the determinant, it is a scalar that tells us how much the volume of a parallelepiped is scaled by when we apply a linear transformation to it. If the determinant is zero, then the volume of the parallelepiped is transformed to zero and therefore the transformation is not invertible. The singularities of Euler angles are of a different nature, where the function we have defined breaks down and fails to be well defined.

### Making a Singularity with Euler Angles

Let's consider the ZYZ sequence Euler angles. If we have the following angles

$$
    \mathbf{\Gamma}
    =
    \begin{pmatrix}
        30^\circ \\
        0 \\
        30^\circ
    \end{pmatrix}
$$

that will give us the following rotation matrix

$$
    \bf{R}(\mathbf{\Gamma})
    =
    \mathbf{R}_z(30^\circ) \ \mathbf{R}_y(0) \ \mathbf{R}_z(30^\circ)
$$

a rotation of $\mathbf{R}_y(0)$ will give the identity matrix. Therefore, the rotation becomes

$$
    \bf{R}(\mathbf{\Gamma})
    =
    \mathbf{R}_z(30^\circ) \ \mathbf{R}_z(30^\circ)
$$

which is equivalent to a rotation of $\mathbf{R}_z(60^\circ)$. Therefore, no matter what the value of our first or third angle is, the rotation is always going to be around the z-axis. When the second angle is zero, this three-angle representation breaks down and we have a singularity.

### Making a Singularity with RPY Angles

Of course, this can occur with any of the three-angle representations. For example, if we have ZYX RPY angles

$$
    \mathbf{\Gamma}
    =
    \begin{pmatrix}
        \alpha \\
        \beta \\
        \gamma
    \end{pmatrix}
    =
    \begin{pmatrix}
        30^\circ \\
        90^\circ \\
        60^\circ
    \end{pmatrix}
$$

then the rotation matrix is

\begin{align*}
    \bf{R}(\mathbf{\Gamma})
    &=
    \mathbf{R}_z(\gamma) \ \mathbf{R}_y(\beta) \ \mathbf{R}_x(\alpha) \\
    &=
    \mathbf{R}_z(60^\circ) \ \mathbf{R}_y(90^\circ) \ \mathbf{R}_x(30^\circ)
\end{align*}

Now, it's best to follow this using a physical coordinate frame or use your right hand to visualise the rotation.

* Our first rotation is about the z-axis -- this is your thumb. The amount of this rotation does not matter for the singularity to occur.
* If we rotate about the y-axis by $90^\circ$ -- this is your middle finger -- then your index finger will be pointing down (in the opposite direction your thumb was pointing before the rotation).

Whats the issue? 

Our last rotation is about the x-axis -- this is your index finger and this is now on the same axis as the z-axis in our first rotation. Therefore, no matter what the value of our first or third angle is, the rotation is always going to be around the original z-axis. Note that the same issue occurs when the second angle is $-90^\circ$.

### When to use Euler Angles in Practice?

In robotics, we rarely use RPY angles because we rarely need to read or communicate orientations between humans. We do need to do a great deal of computations transforming orientations between various coordinate frames. For this use-case, Euler angles are inconvenient, inefficient, and as we just discussed flawed. When orientations need to be read of be intuited, we can visualise the orientation using software, or simply convert to RPY angles to be read.

However, there are some applications where Euler angles are useful such as in aerospace where orientations need to read and communicated between humans. In these scenarios, it is crucial to choose an angle sequence where the singularity is impossible or improbable to occur. With our ZYX RPY angles, the singularity occurs with a pitch angle of $\pm 90^\circ$. This is not a very likely scenario for plane, boat or car as they would have to be facing straight up or straight down.

## Test Your Understanding

#### Question 1

What is the difference between the Euler vector and Euler angles?

#### Question 2

What do Euler angles refer to?

#### Question 3

What is the difference between Euler angles and RPY angles?

#### Question 4

At what angle does the singularity occur for the YZY Euler angles?

#### Question 5

Would the ZYX RPY angles be a good choice of angle representation for a rocket going into space?

#### Question 6

Write a python method that returns a rotation matrix given a set of XYZ Euler angles. Construct the rotation matrix using the `expm` method.


<br>

<a id='6'></a>
# 6.0 Quaternions
---

Quaternions are widely used today for robotics, computer vision, computer graphics and aerospace navigation systems. The quaternion was invented nearly 200 years ago as an extension of the complex number – a hypercomplex number – with a real part and three complex parts

$$
    \check{q} = s + v_x i + v_y j + v_z k \in \mathbb{H}
$$

where the orthogonal complex numbers $i$, $j$ and $k$ are defined such that

$$
    i^2 = j^2 = k^2 = ijk = -1.
$$

_The set of $\mathbb{H}$ represents the set of all hypercomplex numbers that form a quaternion._

It is more useful to think of a quaternion as a scalar value and a vector $\check{q} = (s, \mathbf{v})$ or in component form

$$
    \check{q} = s \langle v_x, \ v_y, \ v_z \rangle
$$

The magnitude of a quaternion is

\begin{align*}
    \lvert\lvert \check{q} \rvert\rvert 
    &= \sqrt{\check{q} \cdot \check{q}} \\
    &= \sqrt{s^2 + v_x^2 + v_y^2 + v_z^2}.
\end{align*}

To represent rotations, we use unit quaternions or versors, quaternions with unit magnitude $\lvert\lvert \check{q} \rvert\rvert = 1$ and denoted by $\mathring{q}$. They can be considered as a rotation of $\theta$ about the unit vector $\hat{\mathbf{v}}$ which are related to the components of the unit quaternion by

\begin{align*}
    \mathring{q}
    &=
    \cos
    \left(
        \frac{\theta}{2}
    \right)
    \left\langle
        \hat{\mathbf{v}}
        \sin
        \left(
            \frac{\theta}{2}
        \right)
    \right\rangle
    \in \mathbf{S}^3
    \subset \mathbb{H}
\end{align*} 

where $\hat{\mathbf{v}}$ is a unit vector that describes the axis of rotation and $\theta$ is the angle of rotation. These are the same four numbers that we used to describe the angle-axis representation of a rotation from earlier

$$
    \hat{\mathbf{\eta}} \theta = \hat{\mathbf{v}} \theta
$$

_The set $\mathbf{S}^3$ is the 3-sphere and represents all angles in 4D space._

It is much better to think of unit quaternions or versors (i.e. the quaternions that represent rotations) as a slight variation to the angle-axis representation rather than as a hyper-complex number in 4D space. Similar to $\mathbf{SO}(3)$, unit quaternions also form a Lie group.

__Note: All Unit Quaternions are valid Quaternions and are also valid 3D rotations. However, not all Quaternions are Unit Quaternions or 3D Rotations.__

Quaternions represent all four dimensional hyper-complex numbers and have many use cases mathematics not limited to representing orientation. However, and confusingly, the term quaternion and unit quaternion are often used interchangeably in the context of describing a rotation.

## Composition of Quaternions

The composition of two unit quaternions $\mathring{q}_1$ and $\mathring{q}_2$ is given by the Hamilton product or quaternion product

$$
    \mathring{q}_1 \circ \mathring{q}_2 =
    s_1 s_2 -
    \mathbf{v}_1 \cdot \mathbf{v}_2
    \langle
        s_1 \mathbf{v}_2 + s_2 \mathbf{v}_1 +
        \mathbf{v}_1 \times \mathbf{v}_2
    \rangle
$$

We can also write this as a matrix multiplication

$$
    \mathring{q}_1 \circ \mathring{q}_2 =
    \begin{pmatrix}
        s_1 & -v_{1_x} & -v_{1_y} & -v_{1_z} \\
        v_{1_x} & s_1 & v_{1_z} & -v_{1_y} \\
        v_{1_y} & v_{1_z} & s_1 & v_{1_x} \\
        v_{1_z} & v_{1_y} & -v_{1_x} & s_1
    \end{pmatrix}
    \begin{pmatrix}
        s_2 \\
        v_{2_x} \\
        v_{2_y} \\
        v_{2_z}
    \end{pmatrix}
$$



Note that the quaternion multiplication is not commutative, i.e. $\mathring{q}_1 \circ \mathring{q}_2 \neq \mathring{q}_2 \circ \mathring{q}_1$. The quaternions $\mathring{q}_1$ and $\mathring{q}_2$ must follow our existing rules for composing rotations with the sub and superscript rules, for example if

$$
    \mathring{q}_1 = {^a \mathring{q}_b} \quad \text{and} \quad \mathring{q}_2 = {^b \mathring{q}_c}
$$

then 

$$
    \mathring{q}_1 \circ \mathring{q}_2 = {^a \mathring{q}_c}
$$

In [30]:
# Lets define two angle axis rotations
# The first is a rotation of 90 degrees about the x-axis
# The second is a rotation of 90 degrees about the y-axis
nhat1 = np.array([1, 0, 0])
nhat2 = np.array([0, 1, 0])
theta1 = np.pi/2
theta2 = np.pi/2

# Now convert this to the Euler vector (also the exponential coordinates)
n1 = theta1 * nhat1
n2 = theta2 * nhat2

ds.display(ds.Math(f"\\eta_1 = {sym.latex(sym.Matrix(np.round(n1, 2)))}"))
ds.display(ds.Math(f"\\eta_2 = {sym.latex(sym.Matrix(np.round(n2, 2)))}"))

# Now convert this to an SO3 rotation matrix
R1 = expm(smb.skew(n1))
R2 = expm(smb.skew(n2))

ds.display(ds.Math(f"R_1 = {sym.latex(sym.Matrix(np.round(R1, 2)))}"))
ds.display(ds.Math(f"R_2 = {sym.latex(sym.Matrix(np.round(R2, 2)))}"))

# Now lets convert the angle axis to a quaternion
q1 = np.array([
    np.cos(theta1/2),
    np.sin(theta1/2) * nhat1[0],
    np.sin(theta1/2) * nhat1[1],
    np.sin(theta1/2) * nhat1[2]
])

q2 = np.array([
    np.cos(theta2/2),
    np.sin(theta2/2) * nhat2[0],
    np.sin(theta2/2) * nhat2[1],
    np.sin(theta2/2) * nhat2[2]
])

ds.display(ds.Math(f"q_1 = {sym.latex(sym.Matrix(np.round(q1, 2)))}"))
ds.display(ds.Math(f"q_2 = {sym.latex(sym.Matrix(np.round(q2, 2)))}"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [31]:
# Lets compose these two matrices
R3 = R1 @ R2

ds.display(ds.Math(f"R_1 R_2 = {sym.latex(sym.Matrix(np.round(R3)))}"))

# Now lets compose these two quaternions using the Hamilton product

q3 = np.array([
    q1[0] * q2[0] - q1[1] * q2[1] - q1[2] * q2[2] - q1[3] * q2[3],
    q1[0] * q2[1] + q1[1] * q2[0] + q1[2] * q2[3] - q1[3] * q2[2],
    q1[0] * q2[2] - q1[1] * q2[3] + q1[2] * q2[0] + q1[3] * q2[1],
    q1[0] * q2[3] + q1[1] * q2[2] - q1[2] * q2[1] + q1[3] * q2[0]
])

ds.display(ds.Math(f"q_1 q_2 = {sym.latex(sym.Matrix(np.round(q3, 2)))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [32]:
# Now lets decompose this quaternion back into an angle axis representation

# First lets get the angle
theta3 = 2 * np.arccos(q3[0])

# Now lets get the axis
nhat3 = q3[1:] / np.sin(theta3/2)

# and then the Euler vector
n3 = theta3 * nhat3

ds.display(ds.Math(f"\\eta_3 \ \\text{{(from quat)}} = {sym.latex(sym.Matrix(np.round(n3, 2)))}"))

# Now lets decompose the rotation matrix back to the Euler vector
n3_from_R = smb.vex(logm(R3))

ds.display(ds.Math(f"\\eta_3 \ \\text{{(from rot)}} = {sym.latex(sym.Matrix(np.round(n3_from_R, 2)))}"))

# Which shows that we get the same result with both methods

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## The Null Rotation

The null rotation or identity for the unit quaternion is given by

$$
    \mathring{q}_0 = 1 \langle 0, \ 0, \ 0 \rangle
$$

In [33]:
# Let's make this quaternion using the UnitQuaternion class of spatialmath
q = sm.UnitQuaternion([1.0, 0.0, 0.0, 0.0])

ds.display(ds.Math(f"q = {sym.latex(sym.Matrix(np.round(q.A, 2)))}"))

theta, nhat = q.angvec()

ds.display(ds.Math(f"\\theta = {theta}"))
ds.display(ds.Math(f"\hat{{\\eta}} = {sym.latex(sym.Matrix(np.round(nhat, 2)))}"))

# Now lets convert this to a rotation matrix
R = expm(smb.skew(nhat * theta))

ds.display(ds.Math(f"R = {sym.latex(sym.Matrix(np.round(R, 2)))}"))

# Which shows that the quaterion [1, 0, 0, 0] is the identity rotation or null rotation

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## The Inverse Rotation

We can take the conjugate of a quaternion $\check{q}$

$$
    \check{q}^* = s \langle -v_x, \ -v_y, \ -v_z \rangle \in \mathbb{H}
$$

with $\check{q}^*$ providing the inverse rotation.

In [34]:
# Make a Unit Quaternion
# This time we will use the UnitQuaternion class from spatialmath
q = sm.UnitQuaternion([1.0, 1.0, 0.0, 0.0])

# Now lets invert this quaternion
# We did this the mannual way, we could have also just done q.inv()
q_inv = sm.UnitQuaternion([q.A[0], -q.A[1], -q.A[2], -q.A[3]])

# Now lets multiply these two quaternions using the Hamilton product (spatialmath provides this for us)
q3 = q * q_inv


ds.display(ds.Math(f"q = {sym.latex(sym.Matrix(np.round(q.A, 2)))}"))
ds.display(ds.Math(f"q^{-1} = {sym.latex(sym.Matrix(np.round(q_inv, 2)))}"))
ds.display(ds.Math(f"q \\circ q^{{-1}} = {sym.latex(sym.Matrix(np.round(q3.A, 2)))}"))

# Which shows that we get the identity quaternion, or null rotation, satisfying the inverse rule

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## The Double Mapping

An interesting property of quaternions is that they represent a _double covering_ of the $\mathbf{SO}(3)$ group. This means that there are two unit quaternions to represent each rotation $\in \mathbf{SO}(3)$. We have the relationship that $\mathring{q}$ and $-\mathring{q}$ represents the same rotation.

## Using Quaternions

Unit quaternions represent yet another way to represent rotations in 3D space.

Compared to other angle representations we have looked at, quaternions have some advantages and disadvantages

* _Angle Axis_ -- Compared to the angle-axis representation, quaternions form a group closed under the Hamilton product. This means that we can compose rotations using the Hamilton product, while to compose rotations using the angle-axis representation we must first convert the angle-axis representation to a rotation matrix and then multiply the resulting rotation matrices.
* $\mathbf{SO}(3)$ -- Compared to the rotation matrix representation, quaternions use only four numbers instead of nine making them more compact and memory efficient. This can be useful when working on low memory or embedded devices. 
    * The Hamilton product uses 16 multiplications and 12 additions
    * Composition of $\mathbf{SO}(3)$ matrices use 27 multiplications and 18 additions
    * Although note that to compute the hamilton product as a matrix multiplication (computers are highly optimised for matrix multiplication) requires the conversion of the first quaternion to a $4 \times 4$ matrix, which will incur extra operations.
* _Euler Angles_ -- Compared to Euler angles, quaternions form a group and are free from the gimbal lock issue.

When implementing quaternions on a computer, they are typically represented as an array with four values. For example, `spatialmath` implements quaternions as a `NumPy` array with four values $s, \ v_x, \ v_y, \ v_z$. Note that other software packages -- notable ROS -- represent quaternions with different ordering of the values, for example $v_x, \ v_y, \ v_z, \ s$. There are also many naming conventions, ROS uses `x, y, z, w` where `w` is the scalar part of the quaternion and `x, y, z` are the vector or imaginary part of the quaternion.


## Test Your Understanding

#### Question 1

What is the difference between the Euler vector and a Unit Quaternion?

#### Question 2

What is the difference between a Quaternion and a Versor?

#### Question 3

What is the difference between a Quaternion and a Unit Quaternion?

#### Question 4

Show that $\mathring{q}$ and $-\mathring{q}$ both result in the same rotation matrix.

#### Question 5

Investigate the computational cost of the Hamilton product for quaternions, the matrix multiplication representation of the Hamilton product, and the composition of $\mathbf{SO}(3)$ matrices. Which is the most efficient? (Hint: the `timeit` or `time` module in Python may be useful)

